Download a bucket from S3 to the local filesystem.

In [1]:
import boto3, os
from timeit import default_timer as timer

In [2]:
def download_dir(client, resource, dist, local='/tmp', bucket='your_bucket'):
    paginator = client.get_paginator('list_objects')
    for result in paginator.paginate(Bucket=bucket, Delimiter='/', Prefix=dist):
        if result.get('CommonPrefixes') is not None:
            for subdir in result.get('CommonPrefixes'):
                download_dir(client, resource, subdir.get('Prefix'), local, bucket)
        if result.get('Contents') is not None:
            for file in result.get('Contents'):
                if not os.path.exists(os.path.dirname(local + os.sep + file.get('Key'))):
                     os.makedirs(os.path.dirname(local + os.sep + file.get('Key')))
                resource.meta.client.download_file(bucket, file.get('Key'), local + os.sep + file.get('Key'))
                print('.', end='')

In [ ]:
bucket_name = ''

In [5]:
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')
buckets = s3_client.list_buckets()
assert 'Buckets' in buckets
buckets = buckets['Buckets']
bucket_names = [bucket['Name'] 
                for bucket in buckets 
                if 'slav-lambdascraper' in bucket['Name']]

In [7]:
root_dir = './data_new'
os.makedirs(root_dir)

    dir_from_bucket = bucket_name.replace('slav-lambdascraper-query-', '')
    bucket_dir = os.path.join(root_dir, dir_from_bucket)
    print('Processing {} '.format(bucket_dir), end='')
    os.makedirs(bucket_dir, exist_ok=True)
    download_dir(s3_client, s3_resource, '', bucket_dir, bucket=bucket_name)
    print('Done in {}s'.format(timer() - start))

Processing ./data_new/alebrije ..................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................Done in 266.6530030090362s
Processing ./data_new/ancient-murals .......................................................................................................................